# A3 Implement a RESTful service

<img src="https://comp421.cs.unc.edu/static/images/COMP421Logo.png" 
     align="right" style="display:inline; width:100px; ">

In this assignment you will implement a simple database application with a REST interface. You'll use the Python [Bottle](https://bottlepy.org/docs/dev/tutorial.html) framework to handle the routing of requests to your code. I'll show you how it is done with an example below.

We're running the server inside this notebook. Of course in the real world you wouldn't do it like this but the code would be nearly the same. Look at the examples in the [REST lecture notes](https://comp421.cs.unc.edu/CH7-InternetReST-Preclass.slides.html) for details. You don't have to do any of that for this assignment.

A bit of explanation about how this is working. You are writing functions that are bound to **routes** by the Bottle framework. The **decorator** above the function definition describes the binding. It looks like magic but under the
covers they are simply building a table that maps strings in URLs to function calls. A simple route might look like this:

```python
@app.route('/foo')        # this is the decorator describing the route
def index():              # this is the function to call
    return {'r': 'Hello'} # return an object with key r and value "Hello"
```

When I am testing your code, I call a function I wrote named `getJson`. `getJson('foo')` uses the Python standard library to submit an HTTP request to `http://localhost:8080/foo`. The Bottle server (running in a background thread) receives the request, finds your function, calls it, and then packages whatever the function returns as a HTTP response. `getJson` gets that response, interprets the response code and decodes the JSON, if any. So, it may look like magic but it is all pretty standard web stuff. The one weird thing here is that the **server** (your code) and the **client** (my test code) are running in the same program. You can see the magic in the `server.py` file included with the assignment.

If you would like to see your server working, simply open another tab in your browser and enter a URL like http://localhost:8080/classes to see the list of classes. I use the [JSONview extension for Chrome](https://chrome.google.com/webstore/detail/jsonview/chklaanhfefbnpoihckbnefhakgolnmc?hl=en) to nicely format the JSON for me but that isn't necessary for this assignment.


## DB Schema

```sql
CREATE TABLE Students (
    sid INTEGER PRIMARY KEY,
    fname TEXT,
    lname TEXT)

CREATE TABLE Courses (
    cid INTEGER PRIMARY KEY,
    name TEXT,
    number TEXT,
    department TEXT,
    credit_hours INTEGER)

CREATE TABLE Enrolled_in (
    sid INTEGER,
    cid INTEGER,
    FOREIGN KEY(sid) REFERENCES Students ON DELETE CASCADE,
    FOREIGN KEY(cid) REFERENCES Courses ON DELETE CASCADE,
    UNIQUE (sid, cid))
```

## Setup

Some text with a pink background will appear below this next cell. It should read:

```
Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.
```

I highly recommend you look at **server.py** and the `getJson` and `makeURL` routines.

In [1]:
%autosave 15
import bottle
import sqlite3

# I have a hacked this so that the server runs in a background thread allowing 
# you go keep working without restarting it. Feel free to look at the code but
# don't change it.
from server import app, getJson, makeURL

import comp421
check, report = comp421.start('A3')

# some text should appear below with a pink background. It looks like an error 
# but the notebook is showing it in pink because it came out on stderr.

Autosaving every 15 seconds


Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.



In [2]:
# fill in your info here
Author = "mkrishn"
Collaborators = ["sanjanam", "namitak"]

In [3]:
# You cannot and should not update this cell
check('Author', Author != 'youronyen', points=1, sort=True)
check('Collaborators', Collaborators != ['list', 'their', 'onyens', 'here'], points=1, sort=True)

Author appears correct
Collaborators appears correct


## Helpers

### Reset DB

This cell defines a function that will reset the DB back to the initial state. This will prevent you from going insane because you can't get the correct answer with a database that is different from mine. Of course you wouldn't do this in the real world.

In [4]:
# You cannot and should not update this cell.  Just execute it

import shutil
import os
import stat

dbName = 'A3.sqlite'

def resetDB():
    """Restore the db to the original state"""
    assert Author != 'youronyen', f'The variable Author has not been set in the cell above'
    assert Collaborators != ["list", "their", "onyens"], f'The variable Collaborators has not been set in the cell above'
    os.chmod('A3Save.sqlite', stat.S_IRUSR)
    shutil.copyfile('A3Save.sqlite', dbName)
    
resetDB()

### Connect to DB

Use this function inside your handler to connect to the database. It will take care of closing it for you when the request is completed.

In [5]:
# You cannot and should not update this cell.  Just execute it
def connect(name):
    '''Return a db connection that will close and commit automatically'''
    conn = sqlite3.connect(name)
    conn.row_factory = sqlite3.Row
    conn.cursor().execute('PRAGMA foreign_keys = ON')
    return conn

### makeURL Helper function

Use this to create the URLs to hand back in the JSON. You just call it like

`makeURL('students', 123)`

and it will create a URL in the right format to return.

# Example

I'll complete this one for you in entirety as an example of what you need to do. Pay close attention to the structure.

### Q0. GET /students List students

When a GET request is received with URL `/students` you should return a JSON object with a single key `students` and value list of students in order by increasing `id`. Like this:

```
{ 'students': [
    {
        "lastName": "Majikes",
        "firstName": "John",
        "id": "http://localhost:8080/students/123"
    }
  ]
}
```

In [6]:
# You cannot and should not update this cell.  Just execute it

@app.route('/students')
def indexStudents():
    """List students"""
    # open the DB; commit and close when we exit the block
    with connect(dbName) as conn:
        # run the sql query to get all the students
        cursor = conn.execute('''SELECT S.sid, S.lname, S.fname
                                   FROM Students AS S
                                  ORDER BY S.sid''')
        # reformat the data
        result = [ { "lastName": lname, "firstName": fname, "id": makeURL('students', sid) }
                   for sid, lname, fname in cursor.fetchall() ]
        # Bottle is going to magically convert this dictionary to JSON
        return { 'students': result }

check('Q0. List students', getJson('students'), points=0)

http response = 200
{'students': [{'lastName': 'Carter',
   'firstName': 'Ashwin',
   'id': '/students/0'},
  {'lastName': 'Velasquez', 'firstName': 'Alexandros', 'id': '/students/1'},
  {'lastName': 'Beach', 'firstName': 'Ainsley', 'id': '/students/2'},
  {'lastName': 'Person', 'firstName': 'Kamile', 'id': '/students/3'},
  {'lastName': 'Buck', 'firstName': 'Maciej', 'id': '/students/4'},
  {'lastName': 'Macfarlane', 'firstName': 'Evalyn', 'id': '/students/5'},
  {'lastName': 'Schroeder', 'firstName': 'Monty', 'id': '/students/6'},
  {'lastName': 'Ryder', 'firstName': 'Kiri', 'id': '/students/7'},
  ...]}
Q0. List students appears correct


127.0.0.1 - - [23/Mar/2023 13:16:38] "get /students HTTP/1.1" 200 9781


# Questions

### Q1. GET /students/123 Return specific student data

Return 404 if the given ID is invalid otherwise your result should look like this:
```
{
    "lastName": "Majikes",
    "firstName": "John",
    "id": "http://localhost:8080/students/3",
    "classes": [
        "http://localhost:8080/classes/1",
        "http://localhost:8080/classes/4"
    ]
}
```

Whenever you produce a list, order it by id.

You can produce a 404 response like this:

```python
return bottle.HTTPResponse(status=404)
```

In [7]:
@app.route('/students/<sid>')
def student(sid):
    """Return students info and classes"""
    with connect(dbName) as conn:
        cursor = conn.execute('''SELECT C.cid
                                   FROM Students AS S, Courses AS C, Enrolled_in AS E
                                   WHERE S.sid = E.sid
                                   AND E.cid = C.cid
                                   AND S.sid=?
                                   ORDER BY C.cid''', (sid,))
        classes = [makeURL('classes', cid) for row in cursor.fetchall() for cid in row]
        if not classes:
            return bottle.HTTPResponse(status = 404)
        cursor = conn.execute('''SELECT S.lname, S.fname
                                    FROM Students AS S
                                    WHERE S.sid=?''', (sid,)).fetchall()[0]
        result={"lastName": cursor[0], "firstName": cursor[1], "id": makeURL('students', sid), "classes": classes}
        return result
        
resetDB()
print('This should return succeed with a HTTP response code of 200\n',
      'along with the dictionary of student with sid=1\n',
      getJson('students', 1))

http response = 200
{'lastName': 'Velasquez',
 'firstName': 'Alexandros',
 'id': '/students/1',
 'classes': ['/classes/1',
  '/classes/2',
  '/classes/6',
  '/classes/10',
  '/classes/18']}
This should return succeed with a HTTP response code of 200
 along with the dictionary of student with sid=1
 {'lastName': 'Velasquez', 'firstName': 'Alexandros', 'id': '/students/1', 'classes': ['/classes/1', '/classes/2', '/classes/6', '/classes/10', '/classes/18']}


127.0.0.1 - - [23/Mar/2023 13:16:39] "get /students/1 HTTP/1.1" 200 159


In [8]:
# You cannot and should not update this cell

resetDB()
check('Q1. Student info', getJson('students', 1), points=2.5)
check('Q1. Non-existent student', getJson('students', -1), points=2.5)

http response = 200
{'lastName': 'Velasquez',
 'firstName': 'Alexandros',
 'id': '/students/1',
 'classes': ['/classes/1',
  '/classes/2',
  '/classes/6',
  '/classes/10',
  '/classes/18']}
Q1. Student info appears correct
http response = 404
Q1. Non-existent student appears correct


127.0.0.1 - - [23/Mar/2023 13:16:39] "get /students/1 HTTP/1.1" 200 159
127.0.0.1 - - [23/Mar/2023 13:16:39] "get /students/-1 HTTP/1.1" 404 0


### Q2. DELETE /students/123

Delete the student and all of their enrollments. Produce a [404 response](https://datatracker.ietf.org/doc/html/rfc7231#section-6.5.4) if the student id is invalid. Produce a [204 response](https://tools.ietf.org/html/rfc7231#section-4.3.5) if successful.

A 404 HTTP response can be create with a `bottle.HTTPResponse(status=404)`.
A 204 response can created with a `bottle.HTTPResponse(status=204)`.

In [9]:
@app.route('/students/<sid>', method="delete")
def deleteStudent(sid):
    """Delete a student and their enrollment"""
    # write your code here
    with connect(dbName) as conn:
        cursor = conn.execute('''SELECT S.sid
                                    FROM Students AS S
                                    WHERE S.sid=?''', (sid,)).fetchall()
        if not cursor:
            return bottle.HTTPResponse(status=404)
        cursor=conn.execute('''DELETE 
                                FROM Students AS S
                                WHERE S.sid=?''', (sid,))
        return bottle.HTTPResponse(status=204)
    
resetDB()
print(' This should succeed with a HTTP response code of 204', 
      getJson('students', 1, method='delete'))
print('This should fail to delete and return a HTTP response code of 404', 
      getJson('students', 1, method='delete'))

http response = 204
 This should succeed with a HTTP response code of 204 204
http response = 404
This should fail to delete and return a HTTP response code of 404 404


127.0.0.1 - - [23/Mar/2023 13:16:40] "delete /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:40] "delete /students/1 HTTP/1.1" 404 0


In [10]:
# You cannot and should not update this cell

resetDB()
# this one should succeed returning a 204
check('Q2. Student delete', getJson('students', 1, method='delete'))
# this one should fail returning 404
check('Q2. Student delete again', getJson('students', 1, method='delete'))

http response = 204
Q2. Student delete appears correct
http response = 404
Q2. Student delete again appears correct


127.0.0.1 - - [23/Mar/2023 13:16:40] "delete /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:40] "delete /students/1 HTTP/1.1" 404 0


### Q3. POST /students create a student

The POST body will be json like this:
```
{
    "lastName": "Majikes",
    "firstName": "John"
}
```
I have included the code below to extract the POST body and convert it from json to a Python dictionary ready
for you to access the values like `body['lastName']`.

Return a response like this:
```
{
    "lastName": "Majikes",
    "firstName": "John",
    "id": "http://localhost:8080/students/123"
}
```

The returned "id" from the insert is in the cursor `lastrowid` field.
That id should be the next id in the table. 

In [11]:
@app.route('/students', method='post')
def addStudent():
    """Insert a new student"""
    with connect(dbName) as conn:
        body = bottle.request.json
        # write your code here
        cursor = conn.execute('''INSERT INTO Students(lname, fname) 
                                    VALUES (?, ?)''', (body['lastName'], body['firstName'],))
        rid = cursor.lastrowid
        result = {"lastName": body['lastName'], "firstName": body['firstName'], "id": makeURL('students', rid)}
        return result
resetDB()
aStudent = { "lastName": "Majikes", "firstName": "John" }
print(f'An insert of student {aStudent["firstName"]} {aStudent["lastName"]} should succeed with a HTTP response code of 200\n',
     getJson('students', method='post', postData=aStudent))

http response = 200
{'lastName': 'Majikes', 'firstName': 'John', 'id': '/students/150'}
An insert of student John Majikes should succeed with a HTTP response code of 200
 {'lastName': 'Majikes', 'firstName': 'John', 'id': '/students/150'}


127.0.0.1 - - [23/Mar/2023 13:16:40] "post /students HTTP/1.1" 200 64


In [12]:
# You cannot and should not update this cell


resetDB()
aStudent = { "lastName": "Majikes", "firstName": "John" }
check('Q3. Insert a new student', 
      getJson('students', method='post', postData=aStudent))

http response = 200
{'lastName': 'Majikes', 'firstName': 'John', 'id': '/students/150'}
Q3. Insert a new student appears correct


127.0.0.1 - - [23/Mar/2023 13:16:41] "post /students HTTP/1.1" 200 64


### Q4. PUT /students/1 update a student

Update the name of an existing student. The PUT body will be json like this:
```
{
    "lastName": "Majikes",
    "firstName": "John"
}
```
Return a 204 response on success and 404 on failure.

In [13]:
@app.route('/students/<sid>', method='put')
def updateStudent(sid):
    """Update student info"""
    with connect(dbName) as conn:
        body = bottle.request.json
        # write your code here
        cursor = conn.execute('''SELECT S.sid
                                    FROM Students AS S
                                    WHERE S.sid=%s''' % (sid)).fetchall()
        if not cursor:
            return bottle.HTTPResponse(status=404)
        cursor=conn.execute('''UPDATE Students
                                    SET fname=?, lname=?
                                    WHERE sid=?''',(body['firstName'], body['lastName'], sid,))
        return bottle.HTTPResponse(status=204)

resetDB()
aStudent = { "lastName": "Majikes", "firstName": "John" }
print("Update a student should succeed with a HTTP response code of 204",
      getJson('students', 1, method='put', postData=aStudent))
print("A bogus student should fail with a HTTP response code of 404",
      getJson('students', 999, method='put', postData=aStudent))

http response = 204
Update a student should succeed with a HTTP response code of 204 204
http response = 404
A bogus student should fail with a HTTP response code of 404 404


127.0.0.1 - - [23/Mar/2023 13:16:41] "put /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:41] "put /students/999 HTTP/1.1" 404 0


In [14]:
# You should not and cannot edit this cell


resetDB()
aStudent = { "lastName": "Majikes", "firstName": "John" }
check('Q4. Update a student', getJson('students', 1, method='put', postData=aStudent))
check('Q4. Update a bogus student', getJson('students', 999, method='put', postData=aStudent))

http response = 204
Q4. Update a student appears correct
http response = 404
Q4. Update a bogus student appears correct


127.0.0.1 - - [23/Mar/2023 13:16:42] "put /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:42] "put /students/999 HTTP/1.1" 404 0


### Q5. GET /classes Return list of classes

Return an object with single key `classes` and value list of objects in class id order like this:
```
{ "classes": [
    {
        "id": "http://localhost:8080/classes/123",
        "name": "Biology",
        "number": "345"
    }
  ]
}
```

In [15]:
@app.route('/classes')
def listClasses():
    """Return a list of classes"""
    # write your code here
    with connect(dbName) as conn:
        cursor = conn.execute('''SELECT C.cid, C.name, C.number
                                    FROM Courses AS C
                                    ORDER BY C.cid''')
        result = [ {"id": makeURL('classes', cid), "name": name, "number": number} 
                 for cid, name, number in cursor.fetchall() ]
        return {'classes': result}
resetDB()
print('A list of the classes is ', getJson('classes'))

http response = 200
{'classes': [{'id': '/classes/0',
   'name': 'Principles of Biology',
   'number': '101'},
  {'id': '/classes/1',
   'name': 'Molecular Biology and Genetics',
   'number': '202'},
  {'id': '/classes/2', 'name': 'Biotransport', 'number': '315'},
  {'id': '/classes/3', 'name': 'Biomedical Solid Mechanics', 'number': '345'},
  {'id': '/classes/4',
   'name': 'Principles of Statistical Inference',
   'number': '600'},
  {'id': '/classes/5',
   'name': 'Introduction to Organic Chemistry I',
   'number': '261'},
  {'id': '/classes/6',
   'name': 'Introduction to Polymer Chemistry',
   'number': '420'},
  {'id': '/classes/7', 'name': 'Compliers', 'number': '520'},
  ...]}
A list of the classes is  {'classes': [{'id': '/classes/0', 'name': 'Principles of Biology', 'number': '101'}, {'id': '/classes/1', 'name': 'Molecular Biology and Genetics', 'number': '202'}, {'id': '/classes/2', 'name': 'Biotransport', 'number': '315'}, {'id': '/classes/3', 'name': 'Biomedical Solid Mech

127.0.0.1 - - [23/Mar/2023 13:16:42] "get /classes HTTP/1.1" 200 1942


In [16]:
# You cannot and should not update this cell

resetDB()
check('Q5. List classes', getJson('classes'), points=5)

http response = 200
{'classes': [{'id': '/classes/0',
   'name': 'Principles of Biology',
   'number': '101'},
  {'id': '/classes/1',
   'name': 'Molecular Biology and Genetics',
   'number': '202'},
  {'id': '/classes/2', 'name': 'Biotransport', 'number': '315'},
  {'id': '/classes/3', 'name': 'Biomedical Solid Mechanics', 'number': '345'},
  {'id': '/classes/4',
   'name': 'Principles of Statistical Inference',
   'number': '600'},
  {'id': '/classes/5',
   'name': 'Introduction to Organic Chemistry I',
   'number': '261'},
  {'id': '/classes/6',
   'name': 'Introduction to Polymer Chemistry',
   'number': '420'},
  {'id': '/classes/7', 'name': 'Compliers', 'number': '520'},
  ...]}
Q5. List classes appears correct


127.0.0.1 - - [23/Mar/2023 13:16:42] "get /classes HTTP/1.1" 200 1942


### Q6. GET /classes/123 Return class info

Return info on a specific class; 404 if it doesn't exist. The result should look like this:

```json
{
    "id": "http://127.0.0.1:8080/classes/1",
    "name": "Molecular Biology and Genetics",
    "department": "BIOL",
    "number": 123,
    "credit": 3,
    "enrollment": 23
}
```

In [17]:
@app.route('/classes/<cid>')
def classInfo(cid):
    """Return info on a single class"""
    # write your code here
    with connect(dbName) as conn:
        cursor = conn.execute('''SELECT C.cid, C.name, C.department, C.number, C.credit_hours, COUNT(E.sid)
                                    FROM Courses AS C, Enrolled_in AS E, Students AS S
                                    WHERE E.sid = S.sid
                                    AND E.cid = C.cid
                                    AND C.cid=%s
                                    GROUP BY C.cid''' % (cid)).fetchone()
        if not cursor:
            return bottle.HTTPResponse(status=404)
        result = {"id": makeURL('classes', cursor[0]), "name": cursor[1], "department": cursor[2], "number": cursor[3], 
                  "credit": cursor[4], "enrollment": cursor[5]}
        return result

resetDB()
print('This call should return class cid=1 ', 
      getJson('classes', 1))
print('This call should return a HTTP response code of 404 because 999 is a bogus class',
      getJson('classes', 999))

http response = 200
{'id': '/classes/1',
 'name': 'Molecular Biology and Genetics',
 'department': 'BIOL',
 'number': '202',
 'credit': 4,
 'enrollment': 25}
This call should return class cid=1  {'id': '/classes/1', 'name': 'Molecular Biology and Genetics', 'department': 'BIOL', 'number': '202', 'credit': 4, 'enrollment': 25}
http response = 404
This call should return a HTTP response code of 404 because 999 is a bogus class 404


127.0.0.1 - - [23/Mar/2023 13:16:43] "get /classes/1 HTTP/1.1" 200 123
127.0.0.1 - - [23/Mar/2023 13:16:43] "get /classes/999 HTTP/1.1" 404 0


In [18]:
# You cannot and should not update this cell

resetDB()
check('Q6. get class info', getJson('classes', 1), points=5)
check('Q6. get bogus class info', getJson('classes', 999), points=5)


http response = 200
{'id': '/classes/1',
 'name': 'Molecular Biology and Genetics',
 'department': 'BIOL',
 'number': '202',
 'credit': 4,
 'enrollment': 25}
Q6. get class info appears correct
http response = 404
Q6. get bogus class info appears correct


127.0.0.1 - - [23/Mar/2023 13:16:44] "get /classes/1 HTTP/1.1" 200 123
127.0.0.1 - - [23/Mar/2023 13:16:44] "get /classes/999 HTTP/1.1" 404 0


### Q8. PUT /classes/123/students/1  Add student 1 to class 123

**Note:** there is no Q7

A [PUT](https://tools.ietf.org/html/rfc7231#section-4.3.4) request to a URL like `/classes/<cid>/students/<sid>` should add the student to the class. It should return status 204 if the student was added, 404 if the student or class id is invalid, or 409 if the student is already in the class.

In [19]:
@app.route('/classes/<cid>/students/<sid>', method="put")
def addStudentToClass(cid, sid):
    """Add student to class"""
    # write your code here
    with connect(dbName) as conn:
        scursor = conn.execute('''SELECT sid
                                    FROM Students
                                    WHERE sid=%s''' % (sid)).fetchall()
        ccursor = conn.execute('''SELECT cid
                                    FROM Courses
                                    WHERE cid=%s''' % (cid)).fetchall()
        if not scursor or not ccursor:
            return bottle.HTTPResponse(status=404)
        cursor = conn.execute('''SELECT S.sid, C.cid
                                    FROM Students AS S, Courses AS C, Enrolled_in AS E
                                    WHERE E.sid = S.sid
                                    AND E.cid = C.cid
                                    AND S.sid=%s
                                    AND C.cid=%s''' % (sid, cid)).fetchall()
        if cursor:
            return bottle.HTTPResponse(status=409)
        cursor = conn.execute('''INSERT INTO Enrolled_in(sid, cid)
                                    VALUES (%s, %s)''' % (sid, cid))
        return bottle.HTTPResponse(status=204)
        

resetDB()
print('This add student sid=1 to class cid=2 should fail. HTTP Response 409',
      getJson('classes', 2, 'students', 1, method="put"))
print('This add should fail as it is a duplicate.  HTTP Response 409',
      getJson('classes', 2, 'students', 1, method="put"))
print('This add should return 404 as the cid is invalid. HTTP Response 404 ',
      getJson('classes', 999, 'students', 1, method="put"))
print('This add should return 404 as the sid is invalid. HTTP Response 404',
      getJson('classes', 1, 'students', 999, method="put"))

http response = 409
This add student sid=1 to class cid=2 should fail. HTTP Response 409 409
http response = 409
This add should fail as it is a duplicate.  HTTP Response 409 409
http response = 404
This add should return 404 as the cid is invalid. HTTP Response 404  404
http response = 404
This add should return 404 as the sid is invalid. HTTP Response 404 404


127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/2/students/1 HTTP/1.1" 409 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/2/students/1 HTTP/1.1" 409 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/1/students/999 HTTP/1.1" 404 0


In [20]:
# You cannot and should not update this cell


resetDB()
check('Q8. add class', getJson('classes', 5, 'students', 1, method="put"), points=8)
check('Q8. add class again', getJson('classes', 5, 'students', 1, method="put"), points=10)
check('Q8. add bogus class', getJson('classes', 999, 'students', 1, method="put"), points=10)
check('Q8. add bogus student', getJson('classes', 1, 'students', 999, method="put"), points=10)

http response = 204
Q8. add class appears correct
http response = 409
Q8. add class again appears correct
http response = 404
Q8. add bogus class appears correct
http response = 404
Q8. add bogus student appears correct


127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/5/students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/5/students/1 HTTP/1.1" 409 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:45] "put /classes/1/students/999 HTTP/1.1" 404 0


### Q10. DELETE /classes/123/1  Drop student 1 from class 123

**NOTE:** there is no Q9

A [DELETE](https://tools.ietf.org/html/rfc7231#section-4.3.5) request to a URL like `/classes/<cid>/<sid>` should drop the student from the class. It should return status 204 if the student was dropped, or 404 if the student or class id is invalid.

In [21]:
@app.route('/classes/<cid>/students/<sid>', method="delete")
def dropStudentFromClass(cid, sid):
    """Drop student from class"""
    # write your code here
    with connect(dbName) as conn:
        scursor = conn.execute('''SELECT sid
                                    FROM Students
                                    WHERE sid=%s''' % (sid)).fetchall()
        ccursor = conn.execute('''SELECT cid
                                    FROM Courses
                                    WHERE cid=%s''' % (cid)).fetchall()
        ecursor = conn.execute('''SELECT S.sid, C.cid
                                    FROM Students AS S, Courses AS C, Enrolled_in AS E
                                    WHERE S.sid = E.sid
                                    AND C.cid = E.cid
                                    AND S.sid=%s
                                    AND C.cid=%s''' % (sid, cid)).fetchall()
        if not scursor or not ccursor or not ecursor:
            return bottle.HTTPResponse(status=404)
        cursor = conn.execute('''DELETE FROM Enrolled_in
                                    WHERE sid=%s
                                    AND cid=%s''' % (sid, cid))
        return bottle.HTTPResponse(status=204)
    
resetDB()

# this should succeed
print('The delete of student cid=1 sid=1 is',
      getJson('classes', 1, 'students', 1, method="delete"),
      ' and should be a 204')
# this should fail because the student is no longer in the class
print('The second delete of student cid=1 sid=1 is',
      getJson('classes', 1, 'students', 1, method="delete"),
      ' and should be 404 as the student no longer exists')
# this should fail because the class doesn't exist
print('The delete of student cid=999 sid=1 is',
      getJson('classes', 999, 'students', 1, method="delete"),
      ' and should be 404 because the class does not exist')
# this should fail because the student doesn't exist
print('The delete of student cid=1 sid=999 is ',
      getJson('classes', 1, 'students', 999, method="delete"),
      ' and should be 404 because the student does not exist')

http response = 204
The delete of student cid=1 sid=1 is 204  and should be a 204
http response = 404
The second delete of student cid=1 sid=1 is 404  and should be 404 as the student no longer exists
http response = 404
The delete of student cid=999 sid=1 is 404  and should be 404 because the class does not exist
http response = 404
The delete of student cid=1 sid=999 is  404  and should be 404 because the student does not exist


127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/999 HTTP/1.1" 404 0


In [22]:
# You can not and should not update this cell.

resetDB()

# this should succeed
check('Q10. drop class', getJson('classes', 1, 'students', 1, method="delete"), points=10)
# this should fail because the student is no longer in the class
check('Q10. drop class again', getJson('classes', 1, 'students', 1, method="delete"), points=10)
# this should fail because the class doesn't exist
check('Q10. drop bogus class', getJson('classes', 999, 'students', 1, method="delete"), points=10)
# this should fail because the student doesn't exist
check('Q10. drop bogus student', getJson('classes', 1, 'students', 999, method="delete"), points=10)

http response = 204
Q10. drop class appears correct
http response = 404
Q10. drop class again appears correct
http response = 404
Q10. drop bogus class appears correct
http response = 404
Q10. drop bogus student appears correct


127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/1 HTTP/1.1" 204 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [23/Mar/2023 13:16:46] "delete /classes/1/students/999 HTTP/1.1" 404 0


# Submit your work

Enter your onyen and collaborators below and then run the cell. You'll get the usual submit button.

In [23]:
# You cannot and should not update this cell
# This cell will produce a summary report of your assignment
if Author == 'youronyen' or Collaborators ==  ['list', 'their', 'onyens', 'here']:
    print("You didn't fill out cell 2!")
    Author = 'John Majikes gets all the points'
    Collaborators = ['John', 'Paul', 'George', 'and', 'Ringo']
assert Author != 'youronyen'
assert Collaborators != ['list', 'their', 'onyens', 'here']
report(Author, Collaborators)

  Collaborators: ['sanjanam', 'namitak']
Report for mkrishn
  21 of 21 appear correct, 100 of 100 points


## Done!
<img src="https://comp421.cs.unc.edu/static/images/restartAndClearOutput.png" width="300" style="float: right" />

Now go back, restart the kernel (menu <font color="green">Kernel</font>-><font color="green">Restart and Clear</font>), and then Shift-Enter your way through the notebook to run all the cells again so you an be sure all your code will work as you expect during grading.

## Saving your work
<img src="https://comp421.cs.unc.edu/static/images/saveAndCheckpoint.png" width="300" style="float: right" />

Now save your work by going to (menu <font color='green'>File</font>-><font color='green'>Save and Checkpoint</font>)

## Submit your notebook

After saving, restarting the kernel, and verifying your report,
you can submit your notebook.
Go to https://comp421.cs.unc.edu/notebook


Note that if you actually saved your work you should not see the leaving site message below.
If you do see the `Leave Site` warning, cancel and save your work again.
<br />
<img src="https://comp421.cs.unc.edu/static/images/leaveSite.png" width="300" style="float: left" />